In [ ]:
!pip install git+https://github.com/pabloppp/pytorch-tools -U

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path
import torch.nn.functional as F
import pytorch_lightning as pl
import torch
from torchtools.optim import RangerLars
from torchtools.lr_scheduler import DelayerScheduler
import os

In [ ]:
class VolcanicDataset(torch.utils.data.IterableDataset):
    
    def __init__(self, base_path, df=None, from_directory = False, is_train=True, target_mean=None, target_std=None, downsample=None):
        """
        :param Path base_path: Path where read the segments.
        :param DataFrame df: Dataframe containing the segments.
        :param bool from_directory: Indicates if you will read the segments directly from the directory and not using df
        :param float target_mean: Mean of the target used for standarization
        :param float target_std: Standard deviation of the target used for standarization
        :param bool downsample: Downsample the dataframe with a specific period determined by the value specified.
        """
        self.df = df
        self.base_path = base_path
        self.delta = 1e-16
        self.is_train = is_train
        self.from_directory = from_directory
        self.target_mean, self.target_std = target_mean, target_std
        self.downsample = downsample
 
        
    def segments_iterable(self):
        
        if self.df is None and self.from_directory:
            segments = os.listdir(self.base_path)
        else:
            segments = self.df.iterrows()
        
        for idx, data in enumerate(segments):
            
            # Obtaining the segment name
            if self.is_train:
                segment = data[1].segment_id
                y = data[1].time_to_eruption
            elif self.from_directory:
                segment = data.split('.')[0]
            else:
                segment = data
            
            # Segment read
            segment_timeseries_path = self.base_path/f'{segment}.csv'
            segment_timeseries = pd.read_csv(segment_timeseries_path)
            segment_timeseries = segment_timeseries.fillna(0) # Fill nan values with zeros meaning no signal
            
            if self.downsample is not None: # Downsampling step
                segment_timeseries = segment_timeseries.reset_index()
                segment_timeseries['index'] = segment_timeseries['index']//self.downsample
                segment_timeseries = segment_timeseries.groupby('index').mean()
            
            # Each segment is standarized with its mean and standar deviation independently (Not the best approach)
            segment_timeseries = (segment_timeseries - segment_timeseries.mean())/(segment_timeseries.std() + self.delta) 
            X = torch.tensor(segment_timeseries.values)
            
            
            if self.is_train and self.df is not None:
                y = (y-self.target_mean)/self.target_std # The target is standarized too
                yield X, y
            else:
                yield X
        
    def __iter__(self):
        return self.segments_iterable()

In [ ]:
class VolcanicDataModule(pl.LightningDataModule):

    def __init__(self, batch_size=32, random_state=123, downsample=None):
        super().__init__()
        self.batch_size = batch_size
        self.random_state = random_state
        self.downsample = downsample

    def prepare_data(self):
        self.path = Path('/kaggle/input/predict-volcanic-eruptions-ingv-oe')
        train_path = self.path/'train.csv'
        self.train = pd.read_csv(train_path)

    def setup(self, stage):
        
        if stage == 'fit':
            
            # split dataset
            train_size = int(len(self.train)*0.8)
            self.train = self.train.sample(frac=1, random_state=self.random_state).reset_index(drop=True)
            train_df_target = self.train.loc[:train_size,'time_to_eruption']
            
            self.train_dataset = VolcanicDataset(self.path/'train', self.train.loc[:train_size,:], target_mean=train_df_target.mean(), target_std=train_df_target.std(), downsample=self.downsample)
            self.valid_dataset = VolcanicDataset(self.path/'train', self.train.loc[train_size:,:], target_mean=train_df_target.mean(), target_std=train_df_target.std(), downsample=self.downsample)

    # return the dataloader for each split
    def train_dataloader(self):
        train_dataset = torch.utils.data.DataLoader(self.train_dataset, batch_size=self.batch_size,num_workers=4)
        return train_dataset

    def val_dataloader(self):
        valid_dataset = torch.utils.data.DataLoader(self.valid_dataset, batch_size=self.batch_size,num_workers=4)
        return valid_dataset
    

In [ ]:
"""
--------------------------- Wavenet model ---------------------------
"""

# from https://www.kaggle.com/hanjoonchoe/wavenet-lstm-pytorch-ignite-ver
class WaveBlock(torch.nn.Module):# 1.34e7

    def __init__(self, in_channels, out_channels, dilation_rates, kernel_size):
        super(WaveBlock, self).__init__()
        self.num_rates = dilation_rates
        self.convs = torch.nn.ModuleList()
        self.filter_convs = torch.nn.ModuleList()
        self.gate_convs = torch.nn.ModuleList()

        self.convs.append(torch.nn.Conv1d(in_channels, out_channels, kernel_size=1))
        dilation_rates = [2 ** i for i in range(dilation_rates)]
        for dilation_rate in dilation_rates:
            self.filter_convs.append(
                torch.nn.Conv1d(out_channels, out_channels, kernel_size=kernel_size, padding=int((dilation_rate*(kernel_size-1))/2), dilation=dilation_rate))
            self.gate_convs.append(
                torch.nn.Conv1d(out_channels, out_channels, kernel_size=kernel_size, padding=int((dilation_rate*(kernel_size-1))/2), dilation=dilation_rate))
            self.convs.append(torch.nn.Conv1d(out_channels, out_channels, kernel_size=1))

    def forward(self, x):
        x = self.convs[0](x)
        res = x
        for i in range(self.num_rates):
            x = torch.tanh(self.filter_convs[i](x)) * torch.sigmoid(self.gate_convs[i](x))
            x = self.convs[i + 1](x)
            res = res + x
        return res

    
class Wavenet(torch.nn.Module):

    def __init__(self):
        super(Wavenet, self).__init__()
        self.wave_block1 = WaveBlock(10, 16, 12, 3)
        self.wave_block2 = WaveBlock(16, 32, 8, 3)
        #self.wave_block3 = WaveBlock(32, 64, 4, 3)
        #self.wave_block4 = WaveBlock(64, 128, 1, 3)
        self.fc = torch.nn.Linear(1920032, 1)

    def forward(self, x):
        x = x.permute(0, 2, 1).float()

        x = self.wave_block1(x)
        x = self.wave_block2(x)
        #x = self.wave_block3(x)
        #x = self.wave_block4(x)
        x = x.permute(0, 2, 1)
        x = x.reshape(x.size(0), -1)
        x = self.fc(x)
        
        return x

"""
--------------------------- 1D Convolutional model ---------------------------
"""

class SimpleConvnet(torch.nn.Module): # 1.17e7

    def __init__(self):
        super(SimpleConvnet, self).__init__()
        self.conv1 = torch.nn.utils.weight_norm(torch.nn.Conv1d(10, 16, 3))
        self.conv2 = torch.nn.utils.weight_norm(torch.nn.Conv1d(16, 32, 3))

        #self.linear1 = torch.nn.utils.weight_norm(torch.nn.Linear(128, 32))
        self.linear2 = torch.nn.utils.weight_norm(torch.nn.Linear(479936, 1))

    def forward(self, x):
        x = x.permute(0, 2, 1).float()

        x = F.relu(self.conv1(x))
        x = torch.nn.AvgPool1d(kernel_size=2)(x)
        #x = torch.nn.Dropout(p=0.3)(x)
        x = F.relu(self.conv2(x))
        x = torch.nn.AvgPool1d(kernel_size=2)(x)
        #x = torch.nn.Dropout(p=0.3)(x)
        
        x = x.reshape(x.size(0), -1)
        #print(x.size())
        #x = self.linear1(x)
        y_pred = self.linear2(x)
        
        return y_pred
    
"""
--------------------------- 1D Convolutional + LSTM model ---------------------------
"""

class ConvLSTM(torch.nn.Module): #2.31e7

    def __init__(self):
        super(ConvLSTM, self).__init__()
        self.conv1 = torch.nn.utils.weight_norm(torch.nn.Conv1d(10, 16, 3))
        self.conv2 = torch.nn.utils.weight_norm(torch.nn.Conv1d(16, 32, 3))
        lstm1 = torch.nn.utils.weight_norm(torch.nn.utils.weight_norm(torch.nn.LSTM(32,64, bidirectional=True), 'weight_ih_l0'), 'weight_hh_l0')
        self.lstm1 = torch.nn.utils.weight_norm(torch.nn.utils.weight_norm(lstm1, 'weight_ih_l0_reverse'), 'weight_hh_l0_reverse')
        self.lstm1.flatten_parameters()

        self.linear1 = torch.nn.utils.weight_norm(torch.nn.Linear(128, 32)) 
        self.linear2 = torch.nn.utils.weight_norm(torch.nn.Linear(32, 1))

    def forward(self, x):
        x = x.permute(0, 2, 1).float()

        x = F.relu(self.conv1(x))
        x = torch.nn.AvgPool1d(kernel_size=2)(x)
        x = torch.nn.Dropout(p=0.3)(x)
        x = F.relu(self.conv2(x))
        x = torch.nn.AvgPool1d(kernel_size=2)(x)
        x = torch.nn.Dropout(p=0.3)(x)
        
        x = x.permute(2, 0, 1)

        x, _ = self.lstm1(x)
        x = torch.nn.Dropout(p=0.3)(x)
        
        
        
        x = x.permute(1, 0, 2)
        x = x[:,-1,:]
        x = x.reshape(x.size(0), -1)

        x = self.linear1(x)
        y_pred = self.linear2(x)
        
        return y_pred

"""
--------------------------- Wrapper model ---------------------------
"""


class ConvNet(pl.LightningModule): 
    
    
    def __init__(self, structure='baseline'):
        super(ConvNet, self).__init__()
        
        if structure=='baseline':
            self.model = SimpleConvnet()
        elif structure=='wavenet':
            self.model = Wavenet()
        elif structure=='convlstm':
            self.model = ConvLSTM()
        
    def forward(self, x):

        return self.model(x)
    
    def configure_optimizers(self):
        #optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        total_epochs = 5
        delay_epochs = int(0.7*total_epochs)
        
        optimizer = RangerLars(self.parameters(), lr=1e-2, betas=(.95,.9), eps=1e-1, weight_decay=1e-2)
        
        base_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, delay_epochs) # delay the scheduler for 10 steps
        delayed_scheduler = DelayerScheduler(optimizer, total_epochs - delay_epochs, base_scheduler)

        return optimizer
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss_func = torch.nn.L1Loss()
        loss = loss_func(y_hat.squeeze(), y) #F.cross_entropy(y_hat, y)
        result = pl.TrainResult(loss)

        result.log('train_loss', loss, prog_bar=True)
        return result
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss_func = torch.nn.L1Loss()
        loss = loss_func(y_hat.squeeze(), y) #F.cross_entropy(y_hat, y)
        # Checkpoint model based on validation loss
        result = pl.EvalResult(checkpoint_on=loss)
        result.val_loss = loss

        return result
    
    def validation_epoch_end(self, validation_step_output_result):
        all_validation_step_loss = validation_step_output_result.val_loss
        all_validation_step_acc = validation_step_output_result.val_acc

        
        validation_step_output_result.log('val_loss', torch.mean(all_validation_step_loss), prog_bar=True)
        
        return validation_step_output_result
    

**Note that the sized of the full connected layers will change if you downsample the dataframe, take care of this**

In [ ]:
data_module = VolcanicDataModule(batch_size=32)
model = ConvNet().float()

early_stop_callback = pl.callbacks.EarlyStopping(
   monitor='val_loss',
   min_delta=0.00,
   patience=3,
   verbose=False,
   mode='min'
)
trainer = pl.Trainer(gpus=1, early_stop_callback=early_stop_callback, max_epochs=5, deterministic=True) 
trainer.fit(model, data_module)

In [ ]:
model = # Load the model

In [ ]:
path = Path('/kaggle/input/predict-volcanic-eruptions-ingv-oe')
test_path = self.path/'test'
sample_submission = pd.read_csv('sample_submission.csv')
test_dataset = VolcanicDataset(test_path, sample_submission, is_train=False)

predictions = []
for x in test_dataset:
    predictions.append(model(x))
    
sample_submission['time_to_eruption'] = predictions
sample_submission.to_csv('submission.csv')

TODO:
* [X] Downsampling timeseries.
* [X] Add LSTM.
* Autoencoder.
* Use datatable for faster reading.

This does not seems to be the best option in this scenario. Maybe a more "traditional" way is better but this can be a starting point.